In [17]:
import json

def get_date_balance(normalized_data, tolerance=0.01):
    with open(normalized_data, 'r') as file:
        data = json.load(file)

    x_start = float('inf')
    x_end = float('-inf')
    date = []
    balance = []
    
    for page in data:
        for word in page['words']:
            x_coordinate = word['geometry'][0][0]

            if x_coordinate < x_start:
                x_start = x_coordinate

            if x_coordinate > x_end:
                x_end = x_coordinate

        for word in page['words']:
            x_coordinate = word['geometry'][0][0]
            y_coordinate = word['geometry'][0][1]

            if abs(x_coordinate - x_start) <= tolerance:
                date.append( y_coordinate)

            if abs(x_coordinate - x_end) <= tolerance:
                balance.append(y_coordinate)

    return date, balance, data


In [20]:
def group_rows(date, data):
    rows = []
    
    for i, soo in enumerate(date):

        if i + 1 < len(date):
            next_y_coord = date[i + 1]
        else:
            next_y_coord = soo 
        row_data = []
        for page in data:
            for word in page['words']:
                item_y_coord = word['geometry'][0][1]

                if item_y_coord < next_y_coord and abs(item_y_coord - soo) <= 0.01 :
                    row_data.append(word['value'])
        
        rows.append({'row_data': row_data})
    
    return rows


In [23]:
date, balance, data = get_date_balance('sample/normalized/mandiri-account_statement.json')

rows = group_rows(date, data)

rows
# for entry in distances:
#     print(f"Date: {entry[0]}, Balance: {entry[1]}, Y-Distance: {entry[2]}")

[{'row_data': ['01/03',
   '28/02',
   '704005145267314219110357473483650A',
   'GA',
   '863,702.00',
   '275,713,469.00']},
 {'row_data': ['GA',
   '863,702.00',
   '275,713,469.00',
   '02/03',
   '01/03',
   '704935145267314219110757179000450A',
   'GA',
   '320,784.00',
   '276,034,253.00']},
 {'row_data': ['GA',
   '320,784.00',
   '276,034,253.00',
   '02/03',
   '01/03',
   'PARRESTDEPISSERAUL',
   'GA',
   '769,845.00',
   '276,803.038.00']},
 {'row_data': ['GA',
   '769,845.00',
   '276,803.038.00',
   '03/03',
   '02/03',
   '704905145267314219106157653654150A',
   'GA',
   '549,175.00',
   '277,352,273.00']},
 {'row_data': ['704905145267314219106157653654150A',
   'GA',
   '549,175.00',
   '277,352,273.00',
   '04/03',
   '03/03',
   'DARMGENENPTRSTESELL',
   'GA',
   '9,555,545.00',
   '286.907,918.00']},
 {'row_data': ['GA',
   '9,555,545.00',
   '286.907,918.00',
   '06/03',
   '05/03',
   'BAEEARETIENPIRSETAAL',
   'GA',
   '399,400.00',
   '287,307,318.00']},
 {'row_da

In [ ]:
normalized_data = 'sample/normalized/mandiri-transaction_inquery.json'
target_page = 0
test = get_date_balance(normalized_data)
test

In [15]:
def main():
